<a href="https://colab.research.google.com/github/yc-chen10/113-1-finance/blob/main/week12%E4%BD%9C%E6%A5%AD_%E7%94%9F%E6%88%90%E5%BC%8FAI%E5%88%86%E6%9E%90%E8%82%A1%E7%A5%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 發送 HTTP 請求以獲取網路上的資料
import requests

# 用來處理和分析結構化數據
import pandas as pd

# 用來處理時間
import datetime as dt

####抓股票資料（4年）

In [ ]:
# "一詮"股票代號
stock_id = '2486'

# 獲取今天的日期（型別：str）
date = dt.date.today().strftime("%Y%m%d")
# date = yyyymmdd

print('今日日期：', date)

# 取得證交所網站資料，帶入網址參數 date：時間，stock_id：股票代號
# https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=20240923&stockNo=2486
stock_data = requests.get(f'https://www.twse.com.tw/rwd/zh/ \
            afterTrading/STOCK_DAY?date={date}&stockNo={stock_id}')

# 將結果轉換 JSON 格式
json_data = stock_data.json()

# 將 JSON 格式轉換爲 Dataframe 格式，
# 方便在 Python 中處理結構化資料
df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])

df

今日日期： 20241204


,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/12/02,"4,380,182","484,081,055",111.00,112.00,109.00,110.50,+1.00,"2,642"
1,113/12/03,"20,118,123","2,388,317,316",112.00,121.50,112.00,120.50,+10.00,"11,597"
2,113/12/04,"20,279,814","2,424,225,272",121.00,123.00,117.00,118.50,-2.00,"16,735"


In [ ]:
def get_stock_data(stock_no, year, month):
    # 將日期格式化為 'yyyymmdd'，並指定為該月份的第一天
    date = f"{year}{str(month).zfill(2)}01"
    url = f"https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={date}&stockNo={stock_no}"

    # 發送HTTP請求並獲取響應
    response = requests.get(url)
    if response.status_code == 200:
        # 將結果轉換為JSON
        data = response.json()
        if data['stat'] == 'OK':
            # 返回數據中的每日交易資料
            return pd.DataFrame(data['data'], columns=data['fields'])
        else:
            print(f"No data for {year}-{month}")
            return None
    else:
        print(f"Failed to fetch data for {year}-{month}")
        return None


In [ ]:
def get_multiple_months(stock_no, start_year, start_month, end_year, end_month):
    all_data = []

    # 循環通過年份和月份來調用API
    current_year = start_year
    current_month = start_month

    while (current_year < end_year) or (current_year == end_year and current_month <= end_month):
        # 獲取單月的股票資料
        monthly_data = get_stock_data(stock_no, current_year, current_month)
        if monthly_data is not None:
            all_data.append(monthly_data)

        # 更新到下一個月份
        if current_month == 12:
            current_month = 1
            current_year += 1
        else:
            current_month += 1

    # 合併所有月份的數據
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return None


In [ ]:
stock_no = '2486'
start_year = 2021
start_month = 9
end_year = 2024
end_month = 9

stock_data = get_multiple_months(stock_no, start_year, start_month, end_year, end_month)

# 打印股票資料
if stock_data is not None:
    print(stock_data)


Failed to fetch data for 2022-1
No data for 2022-5
            日期        成交股數           成交金額     開盤價     最高價    最低價     收盤價  \
0    110/09/01  22,328,010  1,171,695,669   50.20   53.90  50.20   53.00   
1    110/09/02  35,639,973  1,935,066,472   52.80   55.50  52.50   54.70   
2    110/09/03  18,766,743  1,013,755,989   54.50   55.50  53.10   54.50   
3    110/09/06  18,137,809    989,826,334   54.10   55.90  53.30   53.50   
4    110/09/07  18,660,351    941,627,386   53.20   53.60  48.75   50.00   
..         ...         ...            ...     ...     ...    ...     ...   
705  113/09/24   8,628,849    821,923,069   94.20   97.30  93.30   94.60   
706  113/09/25   6,529,733    622,251,538   95.60   96.30  94.60   94.90   
707  113/09/26  29,060,361  2,930,693,598   95.80  103.50  95.10  100.50   
708  113/09/27  29,915,593  3,070,955,106  101.00  106.00  99.90  100.00   
709  113/09/30  14,403,514  1,465,344,231   99.30  104.00  99.00  100.00   

      漲跌價差    成交筆數  
0    +3.10  12,

In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      710 non-null    datetime64[ns]
 1   成交股數    710 non-null    object        
 2   成交金額    710 non-null    object        
 3   開盤價     710 non-null    float64       
 4   最高價     710 non-null    float64       
 5   最低價     710 non-null    float64       
 6   收盤價     710 non-null    float64       
 7   漲跌價差    710 non-null    object        
 8   成交筆數    710 non-null    object        
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 50.0+ KB


#### 時間格式轉換def

In [ ]:
def convert_minguo_to_ad(date_str):
    # 113/09/10 透過「/」分割成清單 [113, 09, 10]
    # 分配給變數 year=113, month=09, day=10
    year, month, day = date_str.split('/')
    ad_year = int(year) + 1911  # 將民國年轉換為西元年
    return f"{ad_year}/{month}/{day}"

# 所有民國日期都轉西元日期 e.g.113/09/10 -> 2024/09/10
stock_data['日期'] = stock_data['日期'].apply(convert_minguo_to_ad)

In [ ]:
# 轉換為 pandas 中的 datetime 格式
stock_data['日期'] = pd.to_datetime(stock_data['日期'], format='%Y/%m/%d')

In [ ]:
stock_data.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,2021-09-01,"22,328,010","1,171,695,669",50.20,53.90,50.20,53.00,+3.10,"12,629"
1,2021-09-02,"35,639,973","1,935,066,472",52.80,55.50,52.50,54.70,+1.70,"19,985"
2,2021-09-03,"18,766,743","1,013,755,989",54.50,55.50,53.10,54.50,-0.20,"10,762"
3,2021-09-06,"18,137,809","989,826,334",54.10,55.90,53.30,53.50,-1.00,"10,030"
4,2021-09-07,"18,660,351","941,627,386",53.20,53.60,48.75,50.00,-3.50,"10,851"


#### 轉數字 numeric

In [ ]:
#確保stock_data 轉成數字
stock_data['收盤價'] = stock_data['收盤價'].apply(pd.to_numeric, errors='coerce')
stock_data['開盤價'] = stock_data['開盤價'].apply(pd.to_numeric, errors='coerce')
stock_data['最高價'] = stock_data['最高價'].apply(pd.to_numeric, errors='coerce')
stock_data['最低價'] = stock_data['最低價'].apply(pd.to_numeric, errors='coerce')

In [ ]:
stock_data.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,2021-09-01,"22,328,010","1,171,695,669",50.2,53.9,50.20,53.0,+3.10,"12,629"
1,2021-09-02,"35,639,973","1,935,066,472",52.8,55.5,52.50,54.7,+1.70,"19,985"
2,2021-09-03,"18,766,743","1,013,755,989",54.5,55.5,53.10,54.5,-0.20,"10,762"
3,2021-09-06,"18,137,809","989,826,334",54.1,55.9,53.30,53.5,-1.00,"10,030"
4,2021-09-07,"18,660,351","941,627,386",53.2,53.6,48.75,50.0,-3.50,"10,851"


In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      710 non-null    datetime64[ns]
 1   成交股數    710 non-null    object        
 2   成交金額    710 non-null    object        
 3   開盤價     710 non-null    float64       
 4   最高價     710 non-null    float64       
 5   最低價     710 non-null    float64       
 6   收盤價     710 non-null    float64       
 7   漲跌價差    710 non-null    object        
 8   成交筆數    710 non-null    object        
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 50.0+ KB


#### GAI 生成報告

In [ ]:
pip install ta FPDF

In [ ]:
import pandas as pd
from ta import momentum, trend
import matplotlib.pyplot as plt
from fpdf import FPDF
import openai

In [ ]:
# 設定 OpenAI API Key
openai.api_key = "YOUR_KEY"

In [ ]:
pip show openai


Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: 


In [ ]:
pip install openai==0.28

In [ ]:
!pip install reportlab yfinance

import pandas as pd
import numpy as np
import yfinance as yf
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [ ]:
df = stock_data.copy()

In [ ]:
# 清理成交股數、成交金額和成交筆數
df["成交股數"] = df["成交股數"].astype(str).str.replace(",", "").astype(int)
df["成交金額"] = df["成交金額"].astype(str).str.replace(",", "").astype(int)
df["成交筆數"] = df["成交筆數"].astype(str).str.replace(",", "").astype(int)

# 先處理 "漲跌價差" 列，將非數字字符替換為 0，然後轉換為 float
df["漲跌價差"] = df["漲跌價差"].replace(r'[^\d.-]', '', regex=True).astype(float)

# 查看處理後的資料
print(df.head())

          日期      成交股數        成交金額   開盤價   最高價    最低價   收盤價  漲跌價差   成交筆數
0 2021-09-01  22328010  1171695669  50.2  53.9  50.20  53.0   3.1  12629
1 2021-09-02  35639973  1935066472  52.8  55.5  52.50  54.7   1.7  19985
2 2021-09-03  18766743  1013755989  54.5  55.5  53.10  54.5  -0.2  10762
3 2021-09-06  18137809   989826334  54.1  55.9  53.30  53.5  -1.0  10030
4 2021-09-07  18660351   941627386  53.2  53.6  48.75  50.0  -3.5  10851


In [ ]:
# 將日期列轉換為日期格式
df['日期'] = pd.to_datetime(df['日期'], format='%y/%m/%d')

# 計算30日移動平均（MA_30）
df['MA_30'] = df['收盤價'].rolling(window=30).mean()

# 計算90日移動平均（MA_90）
df['MA_90'] = df['收盤價'].rolling(window=90).mean()

# 計算RSI（相對強弱指標）
delta = df['收盤價'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

# 計算布林帶
df['MA_BB'] = df['收盤價'].rolling(window=20).mean()
df['Upper_Band'] = df['MA_BB'] + 2 * df['收盤價'].rolling(window=20).std()
df['Lower_Band'] = df['MA_BB'] - 2 * df['收盤價'].rolling(window=20).std()

# 計算成交量移動平均
df['Volume_MA'] = df['收盤價'].rolling(window=20).mean()  # 假設成交量就是收盤價（請根據實際數據調整）

# 顯示結果
print(df)

            日期      成交股數        成交金額    開盤價    最高價    最低價    收盤價  漲跌價差   成交筆數  \
0   2021-09-01  22328010  1171695669   50.2   53.9  50.20   53.0   3.1  12629   
1   2021-09-02  35639973  1935066472   52.8   55.5  52.50   54.7   1.7  19985   
2   2021-09-03  18766743  1013755989   54.5   55.5  53.10   54.5  -0.2  10762   
3   2021-09-06  18137809   989826334   54.1   55.9  53.30   53.5  -1.0  10030   
4   2021-09-07  18660351   941627386   53.2   53.6  48.75   50.0  -3.5  10851   
..         ...       ...         ...    ...    ...    ...    ...   ...    ...   
705 2024-09-24   8628849   821923069   94.2   97.3  93.30   94.6   0.4   7155   
706 2024-09-25   6529733   622251538   95.6   96.3  94.60   94.9   0.3   4833   
707 2024-09-26  29060361  2930693598   95.8  103.5  95.10  100.5   5.6  18478   
708 2024-09-27  29915593  3070955106  101.0  106.0  99.90  100.0  -0.5  21224   
709 2024-09-30  14403514  1465344231   99.3  104.0  99.00  100.0   0.0   9179   

         MA_30       MA_90 

In [ ]:


# 計算技術指標
# 1. 移動平均線（MA）
# 計算30日和90日移動平均
df['MA_30'] = df['收盤價'].rolling(window=30).mean()
df['MA_90'] = df['收盤價'].rolling(window=90).mean()

# 2. 相對強弱指標（RSI）
delta = df['收盤價'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

# 3. 布林帶（Bollinger Bands）
rolling_mean = df['收盤價'].rolling(window=20).mean()
rolling_std = df['收盤價'].rolling(window=20).std()
df['Upper_Band'] = rolling_mean + (rolling_std * 2)
df['Lower_Band'] = rolling_mean - (rolling_std * 2)


# 4. 成交量指標（Volume Analysis）
df['Volume_MA'] = df['成交股數'].replace(",", "").astype(int).rolling(window=20).mean()

print(df[['日期', '收盤價', 'MA_30', 'MA_90', 'RSI', 'Upper_Band', 'Lower_Band', 'Volume_MA']])


            日期    收盤價      MA_30       MA_90        RSI  Upper_Band  \
0   2021-09-01   53.0        NaN         NaN        NaN         NaN   
1   2021-09-02   54.7        NaN         NaN        NaN         NaN   
2   2021-09-03   54.5        NaN         NaN        NaN         NaN   
3   2021-09-06   53.5        NaN         NaN        NaN         NaN   
4   2021-09-07   50.0        NaN         NaN        NaN         NaN   
..         ...    ...        ...         ...        ...         ...   
705 2024-09-24   94.6  95.510000  108.147778  43.307087  102.258294   
706 2024-09-25   94.9  95.223333  108.330000  58.549223  101.758010   
707 2024-09-26  100.5  95.206667  108.544444  79.342723  102.390811   
708 2024-09-27  100.0  95.123333  108.735556  77.880184  103.024748   
709 2024-09-30  100.0  94.990000  108.891111  80.476190  103.718544   

     Lower_Band    Volume_MA  
0           NaN          NaN  
1           NaN          NaN  
2           NaN          NaN  
3           NaN        

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from textwrap import wrap

In [ ]:
def generate_dynamic_report(df, filename="dynamic_investment_report.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(100, height - 50, "Investment Report")

    # Add dynamic analysis based on stock data
    c.setFont("Helvetica", 12)
    y_position = height - 80

    # General stock analysis based on moving averages
    c.drawString(100, y_position, "Stock Analysis Based on Moving Averages:")
    y_position -= 20
    for index, row in df.iterrows():
        if pd.notnull(row['MA_30']) and pd.notnull(row['MA_90']):
            # Format MA_30 and MA_90 to 4 decimal places
            ma_30 = f"{row['MA_30']:.4f}"
            ma_90 = f"{row['MA_90']:.4f}"

            ma_analysis = f"Date: {row['日期']}, Close: {row['收盤價']}, 30-day MA: {ma_30}, 90-day MA: {ma_90}"

            # Wrap the main analysis text
            wrapped_text = wrap(ma_analysis, width=100)
            for line in wrapped_text:
                c.drawString(100, y_position, line)
                y_position -= 15  # Move down for next line
                if y_position < 100:  # Prevent going out of page bounds
                    c.showPage()
                    c.setFont("Helvetica", 12)
                    y_position = height - 50

            # Add the Signal part in a new line (with wrap for the signal part)
            signal_text = "Bearish Signal: 30-day MA < 90-day MA" if row['MA_30'] < row['MA_90'] else "Bullish Signal: 30-day MA > 90-day MA"
            wrapped_signal = wrap(signal_text, width=100)
            for line in wrapped_signal:
                c.drawString(100, y_position, line)
                y_position -= 15
                if y_position < 100:
                    c.showPage()
                    c.setFont("Helvetica", 12)
                    y_position = height - 50

    # Conclusion and recommendations based on analysis
    c.drawString(100, y_position, "Investment Recommendations:")
    y_position -= 20
    c.drawString(100, y_position, "1. If the 30-day MA is below the 90-day MA: watch for further market declines.")
    y_position -= 20
    c.drawString(100, y_position, "2. If the 30-day MA is above the 90-day MA: consider holding or buying.")
    y_position -= 20

    # Save the PDF
    c.save()
    print(f"Report generated: {filename}")

In [ ]:

# 下載生成的 PDF
from google.colab import files

# 使用函數生成 PDF
generate_dynamic_report(df)

files.download("dynamic_investment_report.pdf")

Report generated: dynamic_investment_report.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>